# Purpose of script

The purpose of this script is to aggregate Google API data from daily dfs. 

For example, our current approach (outlined in 'examine_google_trends_API.ipynb') exports the daily Google Trends data as separate .csv files, for each date. We need to aggregate these dates into one df

In [3]:
import os
import pandas as pd
import datetime
import boto3

# General Approach

The general approach should look like this:

1. Connect with AWS
2. Use a for loop to iterate through the files in AWS
- They all either begin with `interest_over_time_` or `interest_by_region_`, followed by a date (e.g., 2020-03-20)
- Read in all the files from AWS, store a local version
- Read in the local version as a pandas df, delete local version
- Append the new pandas df to a running pandas df
3. Export the pandas df, with the name indicating the earliest and latest dates in that df (e.g., `interest_over_time_2020-03-01_2021-02-01.csv`. Export locally, then to AWS, then delete local version. 

Once we make the first compiled version, subsequent compiled versions just need to append to the latest compiled version. 

#### Initialize constants

In [4]:
KEYWORDS = ["covid", "coronavirus", "lockdown", "quarantine", "vaccine"]

In [5]:
AWS_BUCKET = os.environ["AWS_BUCKET"]
AWS_ACCESS = os.environ["AWS_ACCESS"]
AWS_SECRET = os.environ["AWS_SECRET"]

In [15]:
FRESH_SCRAPES_FILENAME = ""
INTEREST_OVER_TIME_FILENAME = "interest_over_time_"
INTEREST_BY_REGION_FILENAME = "interest_by_region_"

LOCAL_SCRAPES_DIR = "../../../tweets/google_API_scrapes/"
AWS_SCRAPES_DIR = "google_API_scrapes/"

LOCAL_TIME_FILENAME = LOCAL_SCRAPES_DIR + INTEREST_OVER_TIME_FILENAME
LOCAL_REGION_FILENAME = LOCAL_SCRAPES_DIR + INTEREST_BY_REGION_FILENAME

AWS_TIME_FILENAME = AWS_SCRAPES_DIR + INTEREST_OVER_TIME_FILENAME
AWS_REGION_FILENAME = AWS_SCRAPES_DIR + INTEREST_BY_REGION_FILENAME

#### Connect with AWS S3 instance

In [7]:
s3 = boto3.client('s3',
                  aws_access_key_id=AWS_ACCESS,
                  aws_secret_access_key=AWS_SECRET)

#### Download one file, as a test

Download one `interest_over_time_` file, as a test

In [9]:
TEST_DATE = "2020-10-15"

In [12]:
AWS_TIME_FILENAME + TEST_DATE + ".csv"

'google_API_scrapes/interest_over_time_2020-10-15.csv'

In [45]:
s3.download_file(Bucket=AWS_BUCKET, 
                 Key= AWS_TIME_FILENAME + TEST_DATE + ".csv", 
                 Filename=LOCAL_TIME_FILENAME + TEST_DATE + ".csv")

In [46]:
time_df = pd.read_csv(LOCAL_TIME_FILENAME + TEST_DATE + ".csv")

In [51]:
time_df.head()

,date,covid,coronavirus,lockdown,quarantine,vaccine,isPartial
0,2020-10-15,100,32,1,4,7,False


In [47]:
os.remove(LOCAL_TIME_FILENAME + TEST_DATE + ".csv")

In [ ]:
# create an aggregate df
time_aggregate_df = pd.DataFrame(columns = ['date', 'covid', 'coronavirus', 
                                       'lockdown', 'quarantine', 
                                       'vaccine','isPartial'])

ASIDE: doing the same steps, but for `region` data

In [48]:
s3.download_file(Bucket=AWS_BUCKET, 
                 Key= AWS_REGION_FILENAME + TEST_DATE + ".csv", 
                 Filename=LOCAL_REGION_FILENAME + TEST_DATE + ".csv")

In [49]:
region_df = pd.read_csv(LOCAL_REGION_FILENAME + TEST_DATE + ".csv")

In [50]:
region_df.head()

,geoName,covid,coronavirus,lockdown,quarantine,vaccine
0,Alabama,75,21,0,1,3
1,Alaska,77,16,0,0,7
2,Arizona,64,29,0,2,5
3,Arkansas,71,21,2,0,6
4,California,69,22,1,2,6


In [52]:
region_df.columns

Index(['geoName', 'covid', 'coronavirus', 'lockdown', 'quarantine', 'vaccine'], dtype='object')

In [57]:
# create an aggregate df
region_aggregate_df = pd.DataFrame(columns = ['geoName', 'covid', 'coronavirus', 
                                              'lockdown', 'quarantine', 'vaccine', 'date'])

In [58]:
# add date column to region_df
region_df["date"] = TEST_DATE

In [59]:
region_df.head()

,geoName,covid,coronavirus,lockdown,quarantine,vaccine,date
0,Alabama,75,21,0,1,3,2020-10-15
1,Alaska,77,16,0,0,7,2020-10-15
2,Arizona,64,29,0,2,5,2020-10-15
3,Arkansas,71,21,2,0,6,2020-10-15
4,California,69,22,1,2,6,2020-10-15


Append to df

In [29]:
aggregate_df = aggregate_df.append(df)

In [30]:
aggregate_df

,date,covid,coronavirus,lockdown,quarantine,vaccine,isPartial
0,2020-10-15,100,32,1,4,7,False


### Loop through dates

Now that we know that this works for one script, let's do this for all the files. We have data from `2020-03-01` to `2021-02-14` (this script was created on `2021-02-15`)

In [36]:
start_date = datetime.date(2020, 3, 1)
end_date = datetime.date(2021, 2, 15)
total_num_dates = (end_date - start_date).days

In [37]:
dates_arr = []

In [38]:
for day in range(total_num_dates):
    new_date = (start_date + datetime.timedelta(days = day)).isoformat()
    dates_arr.append(new_date)

In [39]:
dates_arr[-1]

'2021-02-14'

### Create for-loop script

Create aggregate dfs to hold time and region data

In [80]:
time_aggregate_df = pd.DataFrame(columns = ['date', 'covid', 'coronavirus', 
                                           'lockdown', 'quarantine', 
                                           'vaccine','isPartial'])

region_aggregate_df = pd.DataFrame(columns = ['geoName', 'covid', 'coronavirus', 
                                              'lockdown', 'quarantine', 'vaccine', 'date'])

In [81]:
for date in dates_arr:
    
    try:
        
        print(f"Getting Google API trends data (saved in AWS) for {date}")

        # load from AWS, save locally
        s3.download_file(Bucket=AWS_BUCKET, 
                         Key= AWS_TIME_FILENAME + date + ".csv", 
                         Filename=LOCAL_TIME_FILENAME + date + ".csv")

        s3.download_file(Bucket=AWS_BUCKET, 
                         Key= AWS_REGION_FILENAME + date + ".csv", 
                         Filename=LOCAL_REGION_FILENAME + date + ".csv")
        
        # read as csv
        time_df = pd.read_csv(LOCAL_TIME_FILENAME + date + ".csv")
        region_df = pd.read_csv(LOCAL_REGION_FILENAME + date + ".csv")

        # delete local versions of the file
        os.remove(LOCAL_TIME_FILENAME + date + ".csv")
        os.remove(LOCAL_REGION_FILENAME + date + ".csv")
        
        # for time df, append directly to aggregate. For region df, add 'date' column
        time_aggregate_df = time_aggregate_df.append(time_df)
        
        region_df["date"] = date
        region_aggregate_df = region_aggregate_df.append(region_df)
        
        print(f"Finished appending Google API trends data for {date}")
        
    except Exception as e:
        print("Error in appending Google API trends data")
        print(e)
        print(f"Error occurred for the following date: {date}")

Getting Google API trends data (saved in AWS) for 2020-03-01
Finished appending Google API trends data for 2020-03-01
Getting Google API trends data (saved in AWS) for 2020-03-02
Finished appending Google API trends data for 2020-03-02
Getting Google API trends data (saved in AWS) for 2020-03-03
Finished appending Google API trends data for 2020-03-03
Getting Google API trends data (saved in AWS) for 2020-03-04
Finished appending Google API trends data for 2020-03-04
Getting Google API trends data (saved in AWS) for 2020-03-05
Finished appending Google API trends data for 2020-03-05
Getting Google API trends data (saved in AWS) for 2020-03-06
Finished appending Google API trends data for 2020-03-06
Getting Google API trends data (saved in AWS) for 2020-03-07
Finished appending Google API trends data for 2020-03-07
Getting Google API trends data (saved in AWS) for 2020-03-08
Finished appending Google API trends data for 2020-03-08
Getting Google API trends data (saved in AWS) for 2020-0

In [88]:
time_aggregate_df.tail()

,date,covid,coronavirus,lockdown,quarantine,vaccine,isPartial,Unnamed: 0
0,2021-02-08,100,13,1,2,51,False,NaN
0,2021-02-09,100,12,0,2,57,False,NaN
0,2021-02-10,100,12,1,2,57,False,NaN
0,2021-02-11,100,12,1,2,63,False,NaN
0,2021-02-12,100,11,1,2,62,False,NaN


In [89]:
region_aggregate_df.tail()

,geoName,covid,coronavirus,lockdown,quarantine,vaccine,date
46,Virginia,0,0,0,0,0,2021-02-14
47,Washington,0,0,0,0,0,2021-02-14
48,West Virginia,0,0,0,0,0,2021-02-14
49,Wisconsin,0,0,0,0,0,2021-02-14
50,Wyoming,0,0,0,0,0,2021-02-14


Export joined dfs

In [62]:
AGGREGATE_GOOGLE_DIR = "aggregate_google_API_data/"

In [68]:
# start date
dates_arr[0]

'2020-03-01'

In [67]:
# end date
dates_arr[-1]

'2021-02-14'

In [82]:
LOCAL_AGGREGATE_TIME_PATH = LOCAL_SCRAPES_DIR + "aggregate_interest_over_time_"
LOCAL_AGGREGATE_REGION_PATH = LOCAL_SCRAPES_DIR + "aggregate_interest_by_region_"

AWS_AGGREGATE_TIME_PATH = AGGREGATE_GOOGLE_DIR + "aggregate_interest_over_time_"
AWS_AGGREGATE_REGION_PATH = AGGREGATE_GOOGLE_DIR + "aggregate_interest_by_region_"

In [83]:
DATES_FILENAME = f"{dates_arr[0]}_{dates_arr[-1]}.csv"

In [84]:
DATES_FILENAME

'2020-03-01_2021-02-14.csv'

Export locally

In [85]:
time_aggregate_df.to_csv(LOCAL_AGGREGATE_TIME_PATH + DATES_FILENAME)
region_aggregate_df.to_csv(LOCAL_AGGREGATE_REGION_PATH + DATES_FILENAME)

Export to AWS

In [86]:
s3.upload_file(LOCAL_AGGREGATE_TIME_PATH + DATES_FILENAME, AWS_BUCKET, AWS_AGGREGATE_TIME_PATH + DATES_FILENAME)
s3.upload_file(LOCAL_AGGREGATE_REGION_PATH + DATES_FILENAME, AWS_BUCKET, AWS_AGGREGATE_REGION_PATH + DATES_FILENAME)

Remove local versions

In [87]:
os.remove(LOCAL_AGGREGATE_TIME_PATH + DATES_FILENAME)
os.remove(LOCAL_AGGREGATE_REGION_PATH + DATES_FILENAME)